In [1]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value

class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:   #不管是什麼錯誤
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()
# model_dump 將對象轉化為Dict，之後就可以調用python標準庫序列化json字串，会序列化嵌套对象
# 也可以使用dict(model)将对象转化为字典，但嵌套对象不会被转化为字典。
df = pd.DataFrame(all_sites)
df

行政區域  總數量  可借數量  可還數量                  時間
0        大安區   28     3    25 2024-07-03 13:15:22
1        大安區   21     0    21 2024-07-03 13:15:16
2        大安區   16     0    16 2024-07-03 13:14:15
3        大安區   11    10     1 2024-07-03 13:14:20
4        大安區   16     2    14 2024-07-03 13:15:22
...      ...  ...   ...   ...                 ...
1425  臺大公館校區   30    14     1 2024-07-03 13:14:20
1426  臺大公館校區   20    17     3 2024-07-03 13:15:22
1427  臺大公館校區   24    14     4 2024-07-03 13:15:22
1428  臺大公館校區   40    20    10 2024-07-03 13:15:22
1429  臺大公館校區   18     5    13 2024-07-03 12:59:19

[1430 rows x 5 columns]

##### pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise', ordered=True)[source]
 函式可以將給定的資料分佈到範圍內，範圍也可稱為 bins。
- x：	給定陣列
- bins：	資料分類的標準
- right：	布林型。如果為 True，也包括 bins 最右邊的數字。
- labels：	陣列。bins 的標籤。
- retbins：	布林型。如果是 True，則返回 bins。
- precision：	整數型。儲存和展示 bins 的精度
- ordered：	布林型。如果為 True，則將對結果的標籤進行排序。
- pandas.cut() 

In [2]:
bins = [0,1,4,7,11,100]   #0~1不包含1  1~4不包含4  4~7不包含7  7~11不包含11  11~100不包含100
labels = ['0輛','1~3輛','4~6輛','7~10輛','10輛以上']
df['可借數量']



0        3
1        0
2        0
3       10
4        2
        ..
1425    14
1426    17
1427    14
1428    20
1429     5
Name: 可借數量, Length: 1430, dtype: int64

In [6]:
pd.cut(df['可借數量'],bins = bins, right=False, labels=labels)

0        1~3輛
1          0輛
2          0輛
3       7~10輛
4        1~3輛
        ...  
1425    10輛以上
1426    10輛以上
1427    10輛以上
1428    10輛以上
1429     4~6輛
Name: 可借數量, Length: 1430, dtype: category
Categories (5, object): ['0輛' < '1~3輛' < '4~6輛' < '7~10輛' < '10輛以上']

In [5]:
df['可借'] = pd.cut(df['可借數量'],bins = bins, right=False, labels=labels)
df

行政區域  總數量  可借數量  可還數量                  時間     可借
0        大安區   28     3    25 2024-07-03 13:15:22   1~3輛
1        大安區   21     0    21 2024-07-03 13:15:16     0輛
2        大安區   16     0    16 2024-07-03 13:14:15     0輛
3        大安區   11    10     1 2024-07-03 13:14:20  7~10輛
4        大安區   16     2    14 2024-07-03 13:15:22   1~3輛
...      ...  ...   ...   ...                 ...    ...
1425  臺大公館校區   30    14     1 2024-07-03 13:14:20  10輛以上
1426  臺大公館校區   20    17     3 2024-07-03 13:15:22  10輛以上
1427  臺大公館校區   24    14     4 2024-07-03 13:15:22  10輛以上
1428  臺大公館校區   40    20    10 2024-07-03 13:15:22  10輛以上
1429  臺大公館校區   18     5    13 2024-07-03 12:59:19   4~6輛

[1430 rows x 6 columns]

In [7]:
df.groupby(by=['行政區域','可借'])['可借數量'].count()


C:\Users\han\AppData\Local\Temp\ipykernel_16196\4252408923.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(by=['行政區域','可借'])['可借數量'].count()


行政區域  可借   
中山區   0輛        8
      1~3輛     34
      4~6輛     34
      7~10輛    26
      10輛以上    54
               ..
萬華區   0輛        6
      1~3輛     18
      4~6輛     10
      7~10輛    18
      10輛以上    28
Name: 可借數量, Length: 65, dtype: int64

DataFrame.rename(mapper=None, *, index=None, columns=None, axis=None, copy=None, inplace=False, level=None, errors='ignore')[source]
- Rename columns or index labels.

In [8]:
df1 = df.groupby(by=['行政區域','可借'])[['可借數量']].count()
df1.rename(columns={'可借數量':'站點數量'})    #更改columns名稱

C:\Users\han\AppData\Local\Temp\ipykernel_16196\3955084304.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df1 = df.groupby(by=['行政區域','可借'])[['可借數量']].count()


站點數量
行政區域 可借         
中山區  0輛        8
     1~3輛     34
     4~6輛     34
     7~10輛    26
     10輛以上    54
...          ...
萬華區  0輛        6
     1~3輛     18
     4~6輛     10
     7~10輛    18
     10輛以上    28

[65 rows x 1 columns]